In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

from tqdm import tqdm_notebook

from shapely.geometry import Point, LineString

import shapely.wkt

import os

import matplotlib.pyplot as plt
%matplotlib inline

data_path = '/home/fterroso/data/'

/home/fterroso/python/lib/python3.6/site-packages/geopandas/_compat.py:88: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.8.1-CAPI-1.13.3). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
trips_df= pd.read_csv(os.path.join('data', 'raw_trips.csv'), 
                      index_col=0, 
                      parse_dates='timestamp_o timestamp_d'.split())

In [4]:
trips_df['geom_o']= trips_df['geom_o'].apply(lambda x: shapely.wkt.loads(x))

In [5]:
trips_df['geom_d']= trips_df['geom_d'].apply(lambda x: shapely.wkt.loads(x))

In [6]:
trips_df.head()

,user_id,tw_id_o,timestamp_o,CPRO_o,NPRO_O,CGRUPO_o,NGRUPO_o,POB_GRUPO_o,geom_o,tw_id_d,timestamp_d,CPRO_d,NPRO_d,CGRUPO_d,NGRUPO_d,POB_GRUPO_d,geom_d
0,1258006388,1242014584019529728,2020-03-23 09:05:27+00:00,41,Sevilla,084S,Sevilla (SCD Number 4-A),18.048,POINT (241155.7030033822 4141397.433281829),1242190058716835842,2020-03-23 20:42:43+00:00,41,Sevilla,025S,Alcalá del Río,12.029,POINT (238412.4162063967 4160382.205794079)
1,1258006388,1242190058716835842,2020-03-23 20:42:43+00:00,41,Sevilla,025S,Alcalá del Río,12.029,POINT (238412.4162063967 4160382.205794079),1242239109504151564,2020-03-23 23:57:38+00:00,41,Sevilla,084S,Sevilla (SCD Number 4-A),18.048,POINT (241155.7030033822 4141397.433281829)
2,231380112,1241989424113684481,2020-03-23 07:25:28+00:00,17,Girona,25GI,Besalú y otros municipios,5.231,POINT (970453.5231286159 4687892.360277078),1242064620078104578,2020-03-23 12:24:16+00:00,17,Girona,37GI,Banyoles,19.826,POINT (977549.3669210005 4679816.684196526)
3,231380112,1242070404300226574,2020-03-23 12:47:15+00:00,17,Girona,37GI,Banyoles,19.826,POINT (977549.3669210005 4679816.684196526),1242155003655196674,2020-03-23 18:23:25+00:00,17,Girona,25GI,Besalú y otros municipios,5.231,POINT (970453.5231286159 4687892.360277078)
4,1201548956,1242065686043688967,2020-03-23 12:28:30+00:00,8,Barcelona,097B,Parets del Vallès,19.082,POINT (936126.6582087477 4614643.045649199),1242078693377409025,2020-03-23 13:20:11+00:00,8,Barcelona,030B,Castellterçol y otros municipios,6.165,POINT (921480.9852253482 4635413.947512716)


In [7]:
trips_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 348289 entries, 0 to 348288
Data columns (total 17 columns):
 #   Column       Non-Null Count   Dtype              
---  ------       --------------   -----              
 0   user_id      348289 non-null  int64              
 1   tw_id_o      348289 non-null  int64              
 2   timestamp_o  348289 non-null  datetime64[ns, UTC]
 3   CPRO_o       348289 non-null  int64              
 4   NPRO_O       348289 non-null  object             
 5   CGRUPO_o     348289 non-null  object             
 6   NGRUPO_o     348289 non-null  object             
 7   POB_GRUPO_o  348289 non-null  float64            
 8   geom_o       348289 non-null  object             
 9   tw_id_d      348289 non-null  int64              
 10  timestamp_d  348289 non-null  datetime64[ns, UTC]
 11  CPRO_d       348289 non-null  int64              
 12  NPRO_d       348289 non-null  object             
 13  CGRUPO_d     348289 non-null  object             
 14  NGRU

## Get trips duration

In [8]:
trips_df['time_duration'] = trips_df['timestamp_d'] - trips_df['timestamp_o']

In [9]:
trips_df.head()

,user_id,tw_id_o,timestamp_o,CPRO_o,NPRO_O,CGRUPO_o,NGRUPO_o,POB_GRUPO_o,geom_o,tw_id_d,timestamp_d,CPRO_d,NPRO_d,CGRUPO_d,NGRUPO_d,POB_GRUPO_d,geom_d,time_duration
0,1258006388,1242014584019529728,2020-03-23 09:05:27+00:00,41,Sevilla,084S,Sevilla (SCD Number 4-A),18.048,POINT (241155.7030033822 4141397.433281829),1242190058716835842,2020-03-23 20:42:43+00:00,41,Sevilla,025S,Alcalá del Río,12.029,POINT (238412.4162063967 4160382.205794079),0 days 11:37:16
1,1258006388,1242190058716835842,2020-03-23 20:42:43+00:00,41,Sevilla,025S,Alcalá del Río,12.029,POINT (238412.4162063967 4160382.205794079),1242239109504151564,2020-03-23 23:57:38+00:00,41,Sevilla,084S,Sevilla (SCD Number 4-A),18.048,POINT (241155.7030033822 4141397.433281829),0 days 03:14:55
2,231380112,1241989424113684481,2020-03-23 07:25:28+00:00,17,Girona,25GI,Besalú y otros municipios,5.231,POINT (970453.5231286159 4687892.360277078),1242064620078104578,2020-03-23 12:24:16+00:00,17,Girona,37GI,Banyoles,19.826,POINT (977549.3669210005 4679816.684196526),0 days 04:58:48
3,231380112,1242070404300226574,2020-03-23 12:47:15+00:00,17,Girona,37GI,Banyoles,19.826,POINT (977549.3669210005 4679816.684196526),1242155003655196674,2020-03-23 18:23:25+00:00,17,Girona,25GI,Besalú y otros municipios,5.231,POINT (970453.5231286159 4687892.360277078),0 days 05:36:10
4,1201548956,1242065686043688967,2020-03-23 12:28:30+00:00,8,Barcelona,097B,Parets del Vallès,19.082,POINT (936126.6582087477 4614643.045649199),1242078693377409025,2020-03-23 13:20:11+00:00,8,Barcelona,030B,Castellterçol y otros municipios,6.165,POINT (921480.9852253482 4635413.947512716),0 days 00:51:41


In [10]:
trips_df['time_duration'].value_counts(bins=10, normalize=True)

(-1 days +16:59:26.600000, 0 days 02:23:04.900000]         0.453058
(0 days 02:23:04.900000, 0 days 11:46:43.200000]           0.314443
(0 days 11:46:43.200000, 0 days 21:10:21.500000]           0.118129
(0 days 21:10:21.500000, 1 days 06:33:59.800000]           0.084467
(1 days 06:33:59.800000, 1 days 15:57:38.100000]           0.019745
(-2 days +22:06:31.816999999, -1 days +07:35:48.300000]    0.004086
(1 days 15:57:38.100000, 2 days 01:21:16.400000]           0.002960
(-1 days +07:35:48.300000, -1 days +16:59:26.600000]       0.002547
(2 days 01:21:16.400000, 2 days 10:44:54.700000]           0.000488
(2 days 10:44:54.700000, 2 days 20:08:33]                  0.000078
Name: time_duration, dtype: float64

## Get trip distance

In [11]:
def generate_trip_geom_fn(row):
    p1 =  row['geom_o']
    p2 =  row['geom_d']
    if row['geom_o'].geom_type== 'Polygon':
        p1= row['geom_o'].centroid
    if row['geom_d'].geom_type== 'Polygon':
        p2= row['geom_d'].centroid        
    return LineString([p1, p2])

In [12]:
trips_df['geom'] = trips_df.apply(generate_trip_geom_fn, axis=1)

In [13]:
trips_df['dist_km'] = trips_df['geom'].apply(lambda r: r.length/1000)

In [14]:
trips_df.head()

,user_id,tw_id_o,timestamp_o,CPRO_o,NPRO_O,CGRUPO_o,NGRUPO_o,POB_GRUPO_o,geom_o,tw_id_d,timestamp_d,CPRO_d,NPRO_d,CGRUPO_d,NGRUPO_d,POB_GRUPO_d,geom_d,time_duration,geom,dist_km
0,1258006388,1242014584019529728,2020-03-23 09:05:27+00:00,41,Sevilla,084S,Sevilla (SCD Number 4-A),18.048,POINT (241155.7030033822 4141397.433281829),1242190058716835842,2020-03-23 20:42:43+00:00,41,Sevilla,025S,Alcalá del Río,12.029,POINT (238412.4162063967 4160382.205794079),0 days 11:37:16,LINESTRING (241155.7030033822 4141397.43328182...,19.181950
1,1258006388,1242190058716835842,2020-03-23 20:42:43+00:00,41,Sevilla,025S,Alcalá del Río,12.029,POINT (238412.4162063967 4160382.205794079),1242239109504151564,2020-03-23 23:57:38+00:00,41,Sevilla,084S,Sevilla (SCD Number 4-A),18.048,POINT (241155.7030033822 4141397.433281829),0 days 03:14:55,LINESTRING (238412.4162063967 4160382.20579407...,19.181950
2,231380112,1241989424113684481,2020-03-23 07:25:28+00:00,17,Girona,25GI,Besalú y otros municipios,5.231,POINT (970453.5231286159 4687892.360277078),1242064620078104578,2020-03-23 12:24:16+00:00,17,Girona,37GI,Banyoles,19.826,POINT (977549.3669210005 4679816.684196526),0 days 04:58:48,LINESTRING (970453.5231286159 4687892.36027707...,10.750235
3,231380112,1242070404300226574,2020-03-23 12:47:15+00:00,17,Girona,37GI,Banyoles,19.826,POINT (977549.3669210005 4679816.684196526),1242155003655196674,2020-03-23 18:23:25+00:00,17,Girona,25GI,Besalú y otros municipios,5.231,POINT (970453.5231286159 4687892.360277078),0 days 05:36:10,LINESTRING (977549.3669210005 4679816.68419652...,10.750235
4,1201548956,1242065686043688967,2020-03-23 12:28:30+00:00,8,Barcelona,097B,Parets del Vallès,19.082,POINT (936126.6582087477 4614643.045649199),1242078693377409025,2020-03-23 13:20:11+00:00,8,Barcelona,030B,Castellterçol y otros municipios,6.165,POINT (921480.9852253482 4635413.947512716),0 days 00:51:41,LINESTRING (936126.6582087477 4614643.04564919...,25.415076


In [15]:
trips_df['dist_km'].value_counts(bins=20, normalize=True)

(-1.1099999999999999, 55.458]    0.827540
(55.458, 110.915]                0.062930
(110.915, 166.372]               0.020882
(166.372, 221.829]               0.011935
(277.286, 332.744]               0.011668
(221.829, 277.286]               0.010896
(388.201, 443.658]               0.010356
(332.744, 388.201]               0.009659
(443.658, 499.115]               0.009587
(499.115, 554.573]               0.008843
(554.573, 610.03]                0.003956
(610.03, 665.487]                0.002957
(665.487, 720.944]               0.002659
(776.401, 831.859]               0.002242
(720.944, 776.401]               0.001929
(831.859, 887.316]               0.000873
(887.316, 942.773]               0.000818
(942.773, 998.23]                0.000158
(998.23, 1053.688]               0.000098
(1053.688, 1109.145]             0.000011
Name: dist_km, dtype: float64

In [16]:
trips_df.to_csv(os.path.join('data', 'trips_enriched.csv'))